In [1]:
expt_configs = '/home/ildefonso/Documents/repositories/coding/segmentation/compare-seg-models/experiment_configs'
net_configs = '/home/ildefonso/Documents/repositories/coding/segmentation/compare-seg-models/network_configs/'

In [3]:
import os
os.listdir(expt_configs)

['config_02282018_koumura_bird0_GPU1.ini']

In [2]:
import seg_nets

/home/ildefonso/anaconda3/envs/tcn/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
../learn_curve_keras.py

SyntaxError: invalid syntax (<ipython-input-4-f0b79f9bc905>, line 1)

In [5]:
ls

initial-experiment.ipynb


In [8]:
cd /home/ildefonso/Documents/repositories/coding/segmentation/compare-seg-models

/home/ildefonso/Documents/repositories/coding/segmentation/compare-seg-models


In [15]:
!python make_data.py ./experiment_configs/config_20180406_koumura_bird0_GPU1.ini

/home/ildefonso/anaconda3/envs/tcn/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/112.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/24.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/95.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/38.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/131.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/86.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/80.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/85.wav
making

making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/130.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/118.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/132.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/22.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/45.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/60.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/55.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/2.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/78.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/74.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/17.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/110.wav
m

In [24]:
%run learn_curve_keras.py ./experiment_configs/config_20180406_koumura_bird0_GPU1.ini

ImportError: cannot import name 'reshape_inputs_and_make_masks'

In [75]:
from math import ceil

from keras.models import Model
from keras.layers import Input, TimeDistributed, Add, Multiply
from keras.layers.core import Activation, SpatialDropout1D, Lambda, Reshape, Dense
from keras.layers.convolutional import Conv1D, Conv2D, ZeroPadding1D, Cropping1D
from keras.layers.convolutional import MaxPooling1D, MaxPooling2D, UpSampling1D
from keras.layers import Bidirectional
from keras.layers.recurrent import LSTM


import tensorflow as tf
from keras import backend as K

from keras.activations import relu
from functools import partial

In [61]:
x.get_shape()[0]

Dimension(None)

In [93]:
n_filters_by_layer = (32, 64)
max_len = 19292
num_feat = 152
inputs = Input(shape=(max_len, num_feat, 1))
time_steps = 88
n_classes = 13

x = inputs
#new_shape = (time_steps, num_feat, 1)
#x = Reshape(new_shape)(x)
for n_filters in n_filters_by_layer:
    x = Conv2D(n_filters, kernel_size=(5, 5),
               padding="same", activation='relu')(x)
    x = MaxPooling2D(pool_size=(1, 8),
                     strides=(1, 8))(x)
conv_out_shape = x.get_shape().as_list()
num_hidden = conv_out_shape[2] * conv_out_shape[3]
new_shape = (-1, num_hidden)
x = Reshape(new_shape)(x)
x = Bidirectional(
    LSTM(num_hidden, return_sequences=True, dropout=0.25,
         recurrent_dropout=0.1))(x)
x = TimeDistributed(Dense(n_classes, activation="softmax"))(x)

In [94]:
model = Model(inputs=inputs, outputs=x)

In [95]:
model.layers[-1].output_shape

(None, 19292, 13)

In [18]:
def out_width(in_width, filter_width, stride):
    return ceil(float(in_width - filter_width + 1) / float(stride))

In [25]:
freq_bins_after_pool1 = out_width(152,
                                  8,
                                  8)
freq_bins_after_pool2 = out_width(freq_bins_after_pool1,
                                  8,
                                  8)

In [23]:
freq_bins_after_pool1 * 64

4096

In [22]:
64 * 64

4096

In [26]:
freq_bins_after_pool1

19

In [27]:
freq_bins_after_pool2

2

In [28]:
2 * 64

128